# Preparing the Message Schedule

In [14]:
# Imports
from tutorial_functions import *

# We've gone as far as
1. converting the message
2. initializing the hash values
3. intializing the K Constants
3. shifting and rotation
4. NOW Preparing the message

#### Full Algorithm

In [ ]:
import struct
import math

# Constants for SHA-256




def right_rotate(value, shift):
    return (value >> shift) | (value << (32 - shift)) & 0xFFFFFFFF

def pad_message(message):
    # Convert message to bytes using ASCII encoding
    message_bytes = message.encode('ascii')
    
    # Calculate the length of the original message in bits
    original_length_bits = len(message_bytes) * 8
    
    # Pad the message with a single '1' bit followed by '0' bits
    padded_message = message_bytes + b'\x80'
    while (len(padded_message) * 8) % 512 != 448:
        padded_message += b'\x00'
    
    # Append the length of the original message as a 64-bit big-endian integer
    padded_message += struct.pack('>Q', original_length_bits)
    
    return padded_message



def sha256_compress(block, H):
    W = prepare_message_schedule(block)
    a, b, c, d, e, f, g, h = H

    for t in range(64):
        S1 = right_rotate(e, 6) ^ right_rotate(e, 11) ^ right_rotate(e, 25)
        ch = (e & f) ^ (~e & g)
        temp1 = (h + S1 + ch + K[t] + W[t]) & 0xFFFFFFFF
        S0 = right_rotate(a, 2) ^ right_rotate(a, 13) ^ right_rotate(a, 22)
        maj = (a & b) ^ (a & c) ^ (b & c)
        temp2 = (S0 + maj) & 0xFFFFFFFF

        h = g
        g = f
        f = e
        e = (d + temp1) & 0xFFFFFFFF
        d = c
        c = b
        b = a
        a = (temp1 + temp2) & 0xFFFFFFFF

    H[0] = (H[0] + a) & 0xFFFFFFFF
    H[1] = (H[1] + b) & 0xFFFFFFFF
    H[2] = (H[2] + c) & 0xFFFFFFFF
    H[3] = (H[3] + d) & 0xFFFFFFFF
    H[4] = (H[4] + e) & 0xFFFFFFFF
    H[5] = (H[5] + f) & 0xFFFFFFFF
    H[6] = (H[6] + g) & 0xFFFFFFFF
    H[7] = (H[7] + h) & 0xFFFFFFFF

def sha256(message):
    # Pad the message
    padded_message = pad_message(message)
    
    # Process each 512-bit block
    for i in range(0, len(padded_message), 64):
        block = padded_message[i:i+64]
        sha256_compress(block, H)
    
    # Produce the final hash
    return ''.join(f'{value:08x}' for value in H)

# Example usage

hash_value = sha256(message)
print(hash_value)

### Recap

Our initial message:

In [6]:
message = "Hi!"
message

'Hi!'

We have our message converted from text to ascii to binary:

In [5]:
binary_message = ''.join(format(ord(char), '08b') for char in message)
binary_message


'010010000110100100100001'

We have our padded message.

In [10]:
def pad_binary(binary_message,
               block_size=512,
               length_field=64):

    # Add the '1' bit
    padded = binary_message + '1'

    # Calculate Zeros Needed
    msg_size_on_final_block = (len(padded) % block_size)
    zeros_needed = block_size - msg_size_on_final_block - length_field

    # If there are not enough zeros, add another block w/ zeros
    if zeros_needed < 0:
        zeros_needed += 512
    
    # Otherwise, append the zeros needed
    padded += '0' * zeros_needed

    # Add 64-bit message length to the end, padded with zeros
    msg_length = len(binary_message)
    length_bits = format(msg_length, '064b')
    final_padded = padded + length_bits

    # Print the message length in binary and int
    print(f'Message length (binary): {length_bits}')
    print(f'Message length (int): {msg_length}')

    return final_padded

pad_binary(binary_message)

Message length (binary): 0000000000000000000000000000000000000000000000000000000000011000
Message length (int): 24


'01001000011010010010000110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011000'

We have our H Values

In [1]:
# Initial hash values
H = [
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
]

We have our K Constants

In [2]:
K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]

### One Block At A Time
So everything is processed one block at a time.

Now we prepare our message schedule.

We're going to need the `struct` module.

In [11]:
import struct

Then we need an 64 word (chunk) array called `W`, which is the convention for the messuge scheduling array, all initialized at 0.

In [16]:
W = [0] * 64
print_list_in_rows(W, 16)

0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0


### Populate the First 16 Words
- This loop processes the first 16 words of the input block.
- `block[t * 4:t * 4 + 4]` extracts 4 bytes from the block.
- `struct.unpack('>I', ...)` converts these 4 bytes into a 32-bit integer in big-endian format `('>I')`.
- The resulting integer is stored in W[t].

In [ ]:
for t in range(16):
        W[t] = struct.unpack('>I', block[t * 4:t * 4 + 4])[0]

In [ ]:
def prepare_message_schedule(block):
    W = [0] * 64
    
    for t in range(16, 64):
        s0 = (right_rotate(W[t-15], 7) ^ right_rotate(W[t-15], 18) ^ (W[t-15] >> 3))
        s1 = (right_rotate(W[t-2], 17) ^ right_rotate(W[t-2], 19) ^ (W[t-2] >> 10))
        W[t] = (W[t-16] + s0 + W[t-7] + s1) & 0xFFFFFFFF
    return W